In [42]:
from lib.greedy_circs import *
import qiskit 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit.circuit.library import CXGate
import numpy as np
from lib.dvr1d import *
from lib.utils import *
from lib.vqe import DVR_VQE
from lib.pot_gen import get_pot_cr2

# from lib.local_utils import base_dir, scratch_dir

mol_params = cr2_params
spin = 13
mol_params['name'] += f'_{spin}'

# 16 points
params16 = [5.2, 9]

pot, lims = get_pot_cr2(spin)
dvr_vqe = DVR_VQE(mol_params, pot)
def gen_ansatz_op(mol_params, spin, params16, entangler_map):
    from qiskit.quantum_info import SparsePauliOp
    from qiskit.circuit.library import TwoLocal
    from qiskit.circuit.library import ECRGate, SXGate
    mol_params = mol_params.copy()  # create a copy to avoid changing the original mol_params
    mol_params['name'] += f'_{spin}'
    dvr_options = {
            'type': '1d',
            'box_lims': (params16[0], params16[1]),
            'dx': (params16[1] - params16[0]) / 64,
            'count': 64,
        }
    
    # obtain the potential for a CR2 at certain radii
    pot, lims = get_pot_cr2(spin)

    # perform a dvr vqe to obtain the hamiltonian
    dvr_vqe = DVR_VQE(mol_params, pot)
    h_dvr = dvr_vqe.get_h_dvr(dvr_options, J=0) * hartree

     # Perform a Pauli Decomposition to get the Hamiltonian and get a composition.
    h_dvr_p0 = SparsePauliOp.from_operator(h_dvr)
    print(h_dvr_p0.coeffs)
    num_qubits = int(np.log2(h_dvr.shape[0]))
    a = TwoLocal(num_qubits, rotation_blocks=['ry'], entanglement_blocks=['cx'], entanglement='linear', reps=2, insert_barriers=False).decompose()
    a = transpile(a, basis_gates=['ecr',  'x', 'rz', 'sx', 'id'])
    return h_dvr, h_dvr_p0, a


h_dvr, h_dvr_p0, a = gen_ansatz_op(mol_params, spin, params16, entangler_map)



[ 2.8  3.2  3.6  4.   4.4  4.8  5.2  5.6  6.   6.4  6.8  7.2  7.6  8.
  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.  19.  20.  22.  24.
 28.  32.  36.  40. ]
[ 2.8  3.2  3.6  4.   4.4  4.8  5.2  5.6  6.   6.4  6.8  7.2  7.6  8.
  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.  19.  20.  22.  24.
 28.  32.  36.  40. ]
[ 1.78727732e+03+0.j -1.30499463e+03+0.j  4.97595088e+00+0.j ...
  1.95627038e+00+0.j  8.68763976e-07+0.j  9.16096900e-02+0.j]


In [43]:
a.draw()

┌────┐┌──────────────┐┌────┐┌────────┐ ░ ┌──────────┐                »
q_0: ┤ √X ├┤ Rz(θ[0] + π) ├┤ √X ├┤ Rz(3π) ├─░─┤ Rz(-π/2) ├────────────────»
     ├────┤├──────────────┤├────┤├────────┤ ░ └┬────────┬┘┌────┐┌────────┐»
q_1: ┤ √X ├┤ Rz(θ[1] + π) ├┤ √X ├┤ Rz(3π) ├─░──┤ Rz(-π) ├─┤ √X ├┤ Rz(-π) ├»
     ├────┤├──────────────┤├────┤├────────┤ ░  ├────────┤ ├────┤├────────┤»
q_2: ┤ √X ├┤ Rz(θ[2] + π) ├┤ √X ├┤ Rz(3π) ├─░──┤ Rz(-π) ├─┤ √X ├┤ Rz(-π) ├»
     ├────┤├──────────────┤├────┤├────────┤ ░  ├────────┤ ├────┤├────────┤»
q_3: ┤ √X ├┤ Rz(θ[3] + π) ├┤ √X ├┤ Rz(3π) ├─░──┤ Rz(-π) ├─┤ √X ├┤ Rz(-π) ├»
     ├────┤├──────────────┤├────┤├────────┤ ░  ├────────┤ ├────┤├────────┤»
q_4: ┤ √X ├┤ Rz(θ[4] + π) ├┤ √X ├┤ Rz(3π) ├─░──┤ Rz(-π) ├─┤ √X ├┤ Rz(-π) ├»
     ├────┤├──────────────┤├────┤├────────┤ ░  ├────────┤ ├────┤├────────┤»
q_5: ┤ √X ├┤ Rz(θ[5] + π) ├┤ √X ├┤ Rz(3π) ├─░──┤ Rz(-π) ├─┤ √X ├┤ Rz(-π) ├»
     └────┘└──────────────┘└────┘└────────┘ ░  └────────┘ └────┘└────────┘»
«     ┌──────┐   ┌───┐                                                    »
«q_0: ┤0     ├───┤ X ├────────────────────────────────────────────────────»
«     │  Ecr │┌──┴───┴───┐┌──────┐   ┌───┐                                »
«q_1: ┤1     ├┤ Rz(-π/2) ├┤0     ├───┤ X ├────────────────────────────────»
«     └──────┘└──────────┘│  Ecr │┌──┴───┴───┐┌──────┐   ┌───┐            »
«q_2: ────────────────────┤1     ├┤ Rz(-π/2) ├┤0     ├───┤ X ├────────────»
«                         └──────┘└──────────┘│  Ecr │┌──┴───┴───┐┌──────┐»
«q_3: ────────────────────────────────────────┤1     ├┤ Rz(-π/2) ├┤0     ├»
«                                             └──────┘└──────────┘│  Ecr │»
«q_4: ────────────────────────────────────────────────────────────┤1     ├»
«                                                                 └──────┘»
«q_5: ────────────────────────────────────────────────────────────────────»
«                                                                         »
«                               ░ ┌────┐ ┌──────────────┐┌────┐┌────────┐ ░ »
«q_0: ──────────────────────────░─┤ √X ├─┤ Rz(θ[6] + π) ├┤ √X ├┤ Rz(3π) ├─░─»
«                               ░ ├────┤ ├──────────────┤├────┤├────────┤ ░ »
«q_1: ──────────────────────────░─┤ √X ├─┤ Rz(θ[7] + π) ├┤ √X ├┤ Rz(3π) ├─░─»
«                               ░ ├────┤ ├──────────────┤├────┤├────────┤ ░ »
«q_2: ──────────────────────────░─┤ √X ├─┤ Rz(θ[8] + π) ├┤ √X ├┤ Rz(3π) ├─░─»
«        ┌───┐                  ░ ├────┤ ├──────────────┤├────┤├────────┤ ░ »
«q_3: ───┤ X ├──────────────────░─┤ √X ├─┤ Rz(θ[9] + π) ├┤ √X ├┤ Rz(3π) ├─░─»
«     ┌──┴───┴───┐┌──────┐┌───┐ ░ ├────┤┌┴──────────────┤├────┤├────────┤ ░ »
«q_4: ┤ Rz(-π/2) ├┤0     ├┤ X ├─░─┤ √X ├┤ Rz(θ[10] + π) ├┤ √X ├┤ Rz(3π) ├─░─»
«     └──────────┘│  Ecr │└───┘ ░ ├────┤├───────────────┤├────┤├────────┤ ░ »
«q_5: ────────────┤1     ├──────░─┤ √X ├┤ Rz(θ[11] + π) ├┤ √X ├┤ Rz(3π) ├─░─»
«                 └──────┘      ░ └────┘└───────────────┘└────┘└────────┘ ░ »
«     ┌──────────┐                ┌──────┐   ┌───┐                        »
«q_0: ┤ Rz(-π/2) ├────────────────┤0     ├───┤ X ├────────────────────────»
«     └┬────────┬┘┌────┐┌────────┐│  Ecr │┌──┴───┴───┐┌──────┐   ┌───┐    »
«q_1: ─┤ Rz(-π) ├─┤ √X ├┤ Rz(-π) ├┤1     ├┤ Rz(-π/2) ├┤0     ├───┤ X ├────»
«      ├────────┤ ├────┤├────────┤└──────┘└──────────┘│  Ecr │┌──┴───┴───┐»
«q_2: ─┤ Rz(-π) ├─┤ √X ├┤ Rz(-π) ├────────────────────┤1     ├┤ Rz(-π/2) ├»
«      ├────────┤ ├────┤├────────┤                    └──────┘└──────────┘»
«q_3: ─┤ Rz(-π) ├─┤ √X ├┤ Rz(-π) ├────────────────────────────────────────»
«      ├────────┤ ├────┤├────────┤                                        »
«q_4: ─┤ Rz(-π) ├─┤ √X ├┤ Rz(-π) ├────────────────────────────────────────»
«      ├────────┤ ├────┤├────────┤                                        »
«q_5: ─┤ Rz(-π) ├─┤ √X ├┤ Rz(-π) ├────────────────────────────────────────»
«      └────────┘ └────┘└────────┘                                        »
«                          

In [45]:
def Run_VQE(a, optimizers, params, estimator, operator):
        """ This is the function that runs the VQE."""
        repeat = 3
        params = None
        # params = np.array([float(i) for i in result.optimal_parameters.values()])
        converge_cnts1 = np.empty([len(optimizers)], dtype=object)
        converge_vals1 = np.empty([len(optimizers)], dtype=object)

        for i, optimizer in enumerate(optimizers):
            print('Optimizer: {}        '.format(type(optimizer).__name__))
            # algorithm_globals.random_seed = 42

            def store_intermediate_result(eval_count, parameters, mean, std):
                counts.append(eval_count)
                values.append(mean)
                print(f'\r{eval_count}, {mean}', end='')
            best_res1 = None
            
            for j in range(repeat):
                counts = []
                values = []
                vqe = VQE(estimator=estimator, ansatz=a, optimizer=optimizer, initial_point=params, callback=store_intermediate_result)
                results = vqe.compute_minimum_eigenvalue(operator=operator)
                print()
                if (best_res1 is None) or (values[-1] <= best_res1):
                    best_res1 = values[-1]
                    converge_cnts1[i] = np.asarray(counts)
                    converge_vals1[i] = np.asarray(values)
        print('\nOptimization complete ') 
        return converge_cnts1, converge_vals1, results

In [46]:
from qiskit_ibm_runtime import QiskitRuntimeService, Batch, Estimator
from qiskit.algorithms.optimizers import SPSA, COBYLA, NELDER_MEAD, GSLS
from qiskit_aer import AerSimulator
a = transpile(a, basis_gates=['ecr', 'rz', 'x', 'sx', 'id'])
optimizers = [[SPSA(maxiter=1000), COBYLA(maxiter=1000), NELDER_MEAD(maxiter=1000), GSLS(maxiter=10000)]]
jobs = []
service = QiskitRuntimeService(
    channel='ibm_quantum',
    token='50e4f60f02cd247a763d93cbeb949668b4383e6df68ba8a7b4c97b35be7d9cf41f83566cb249b7619d46f081d8349d911392a9d12def4088966310b9168a7f10'
)
backend = service.get_backend("simulator_statevector")

with Batch(backend=backend) as batch:
    estimator = Estimator(batch)
    for opt in optimizers:
        job = Run_VQE(a, opt, None, estimator, h_dvr_p0)
        print(job)
        jobs.append(job)
        

Optimizer: SPSA        
2051, -379.25692201265126
1597, -456.82085428853394

In [10]:
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.compiler import transpile
qc = QuantumCircuit(1)
qc.ry(Parameter('theta'), 0)
qc.measure_all()


qc = transpile(qc,u(), basis_gates=['ecr', 'rz', 'x', 'sx', 'id'])

In [11]:
qc.draw()

global phase: 3π/2
        ┌────┐┌───────────────┐┌────┐┌────────┐ ░ ┌─┐
     q: ┤ √X ├┤ Rz(theta + π) ├┤ √X ├┤ Rz(3π) ├─░─┤M├
        └────┘└───────────────┘└────┘└────────┘ ░ └╥┘
meas: 1/═══════════════════════════════════════════╩═
                                                   0